In [1]:
import pandas as pd
import datetime
etftempdata = pd.read_csv('NIFTYBEES.csv')

In [2]:
etfdata= pd.DataFrame(etftempdata[::-1].reset_index())
etfdata.drop(etfdata.columns[[0]],axis=1,inplace=True)
etfdata

,Date,Close,Open,High,Low,Vol.,Change %
0,"Jan 09, 2002",10.91,11.35,11.35,10.80,6.59M,-2.15%
1,"Jan 10, 2002",10.90,10.90,10.98,10.85,734.35K,-0.09%
2,"Jan 11, 2002",10.81,10.90,10.98,10.76,738.44K,-0.83%
3,"Jan 14, 2002",11.06,10.99,11.14,10.96,370.13K,2.31%
4,"Jan 15, 2002",10.91,10.97,11.07,10.89,385.91K,-1.36%
...,...,...,...,...,...,...,...
4963,"Jan 24, 2022",185.79,196.80,196.80,184.05,9.71M,-2.54%
4964,"Jan 25, 2022",187.07,185.49,187.38,183.05,5.88M,0.69%
4965,"Jan 27, 2022",185.33,194.00,194.00,182.70,5.95M,-0.93%
4966,"Jan 28, 2022",185.10,187.75,188.06,184.80,2.77M,-0.12%


In [3]:
for i in etfdata['Date']:
    datetime.datetime.strptime(i,'%b %d, %Y')

In [4]:
l=[[]]
for i in list(etfdata['Date']):
    z = list(etfdata['Date'].loc[lambda x:x==i].index)
    etfdata.loc[z[0],'Weekday'] = datetime.datetime.strptime(i,'%b %d, %Y').weekday()

etfdata
#     if datetime.datetime.strptime(i,).weekday() == 0:
#         l.append([0])
#     else:
#         l[len(l)-1].append(datetime.datetime.strptime(i,).weekday())

# # print(l)
# for x in l:
#     if len(x) == 5:
#         continue
#     else:
#         print(x)

,Date,Close,Open,High,Low,Vol.,Change %,Weekday
0,"Jan 09, 2002",10.91,11.35,11.35,10.80,6.59M,-2.15%,2.0
1,"Jan 10, 2002",10.90,10.90,10.98,10.85,734.35K,-0.09%,3.0
2,"Jan 11, 2002",10.81,10.90,10.98,10.76,738.44K,-0.83%,4.0
3,"Jan 14, 2002",11.06,10.99,11.14,10.96,370.13K,2.31%,0.0
4,"Jan 15, 2002",10.91,10.97,11.07,10.89,385.91K,-1.36%,1.0
...,...,...,...,...,...,...,...,...
4963,"Jan 24, 2022",185.79,196.80,196.80,184.05,9.71M,-2.54%,0.0
4964,"Jan 25, 2022",187.07,185.49,187.38,183.05,5.88M,0.69%,1.0
4965,"Jan 27, 2022",185.33,194.00,194.00,182.70,5.95M,-0.93%,3.0
4966,"Jan 28, 2022",185.10,187.75,188.06,184.80,2.77M,-0.12%,4.0


Getting Dates list for every unique month

In [5]:
l={}
for i in list(etfdata['Date']):
    # z = list(etfdata['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%b %d, %Y').month,datetime.datetime.strptime(i,'%b %d, %Y').year) in l.keys():
        l[datetime.datetime.strptime(i,'%b %d, %Y').month,datetime.datetime.strptime(i,'%b %d, %Y').year].append(i)
    else:
        l[datetime.datetime.strptime(i,'%b %d, %Y').month,datetime.datetime.strptime(i,'%b %d, %Y').year] = [i]
    



Identifying Expiry Dates and adding to Database

In [6]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
expirySIP = pd.DataFrame(columns=my_columns)
import math
SIPamount = 5000
expiry = {}
for i in l.keys():
    for y in l[i][::-1]:
        if datetime.datetime.strptime(y,'%b %d, %Y').weekday() == 3:
            z = list(etfdata['Date'].loc[lambda x:x==y].index)
            if math.isnan(etfdata.loc[z[0],'Close']):
                date = l[i][l[i].index(y)-1]
                price = etfdata.loc[z[0]-1,'Close']
                qty = math.floor(SIPamount/price)
                expirySIP = expirySIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(y,'%b %d, %Y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            else:
                date = y
                price = etfdata.loc[z[0],'Close']
                qty = math.floor(SIPamount/price)
                expirySIP = expirySIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(y,'%b %d, %Y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            break
expirySIP.to_csv('SIP Details/Expiry SIP.csv')

Normal 1st Trading Day SIP Plan to database

In [ ]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
normalSIP = pd.DataFrame(columns=my_columns)
for i in l.keys():
    for y in l[i]:
        date = datetime.datetime.strptime(y,'%b %d, %Y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            print(y)
            if math.isnan(etfdata.loc[z[0]+1,'Close']):
                continue
            # else:
                # print(y)
        else:
            date = y
            # print(date)
            price = etfdata.loc[z[0],'Close']
            qty = math.floor(SIPamount/price)
            normalSIP = normalSIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(y,'%b %d, %Y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            break
normalSIP.to_csv('SIP Details/Normal SIP.csv')

SIP on the Month Low

In [ ]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
minSIP = pd.DataFrame(columns=my_columns)
for i in l.keys():
    min = 0
    mindate = ''
    for y in l[i]:
        # date = datetime.datetime.strptime(y,'%b %d, %Y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        price = etfdata.loc[z[0],'Close']
        
        if math.isnan(price):
            continue
        elif min == 0:
            min = price
            mindate = y
        elif price<min:
            min = price
            mindate = y
        else:
            continue
    qty = math.floor(SIPamount/min)
    minSIP = minSIP.append(
                    pd.Series(
                        [
                            mindate,
                            datetime.datetime.strptime(mindate,'%b %d, %Y').weekday(),
                            min,
                            qty,
                            qty*min
                        ],index=my_columns
                    ),ignore_index=True
                )

minSIP.to_csv('SIP Details/Min SIP.csv')
        


SIP on Maximum Number

In [ ]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
maxSIP = pd.DataFrame(columns=my_columns)
for i in l.keys():
    max = 0
    maxdate = ''
    for y in l[i]:
        # date = datetime.datetime.strptime(y,'%b %d, %Y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        price = etfdata.loc[z[0],'Close']
        
        if math.isnan(price):
            continue
        elif max == 0:
            max = price
            maxdate = y
        elif price>max:
            max = price
            maxdate = y
        else:
            continue
    qty = math.floor(SIPamount/max)
    maxSIP = maxSIP.append(
                    pd.Series(
                        [
                            maxdate,
                            datetime.datetime.strptime(maxdate,'%b %d, %Y').weekday(),
                            max,
                            qty,
                            qty*max
                        ],index=my_columns
                    ),ignore_index=True
                )

maxSIP.to_csv('SIP Details/Max SIP.csv')

Getting Dates list for every unique Week

In [ ]:
w={}
for i in list(etfdata['Date']):
    # z = list(etfdata['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%b %d, %Y').isocalendar()[1],datetime.datetime.strptime(i,'%b %d, %Y').year) in w.keys():
        w[datetime.datetime.strptime(i,'%b %d, %Y').isocalendar()[1],datetime.datetime.strptime(i,'%b %d, %Y').year].append(i)
    else:
        w[datetime.datetime.strptime(i,'%b %d, %Y').isocalendar()[1],datetime.datetime.strptime(i,'%b %d, %Y').year] = [i]
# print(w)


SIP Every Week

In [ ]:
my_columns = ['Date','Week','Buy Price','Qty','Investment Amount']
weekSIP = pd.DataFrame(columns=my_columns)
for z in w.keys():
    weeks = datetime.datetime(z[1],12,27).isocalendar()[1]
    WeeklySIP = (SIPamount*12)/weeks
    for y in w[z]:
        date = datetime.datetime.strptime(y,'%b %d, %Y')
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            continue
        else:
            price = etfdata.loc[z[0],'Close']
            date = y
            qty = math.floor(WeeklySIP/price)
            weekSIP = weekSIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(date,'%b %d, %Y').isocalendar()[1],
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
            break
weekSIP.to_csv('SIP Details/Week SIP.csv')

Getting Trading Dates list for every unique Year

In [ ]:
d={}
for i in list(etfdata['Date']):
    # z = list(etfdata['Date'].loc[lambda x:x==i].index)
    if (datetime.datetime.strptime(i,'%b %d, %Y').year) in d.keys():
        date = datetime.datetime.strptime(i,'%b %d, %Y')
        z = list(etfdata['Date'].loc[lambda x:x==i].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            continue
        else:
            d[datetime.datetime.strptime(i,'%b %d, %Y').year].append(i)
    else:
        date = datetime.datetime.strptime(i,'%b %d, %Y')
        z = list(etfdata['Date'].loc[lambda x:x==i].index)
        if math.isnan(etfdata.loc[z[0],'Close']):
            continue
        else:
            d[datetime.datetime.strptime(i,'%b %d, %Y').year] = [i]


SIP every day

In [ ]:
my_columns = ['Date','Day','Buy Price','Qty','Investment Amount']
daySIP = pd.DataFrame(columns=my_columns)
for i in d.keys():
    totalDays = len(d[i])
    lastMonth = datetime.datetime.strptime(d[i][-1],'%b %d, %Y').month
    investmentAmount = (SIPamount*lastMonth)/totalDays
    for y in d[i]:
        z = list(etfdata['Date'].loc[lambda x:x==y].index)
        date = y
        price = etfdata.loc[z[0],'Close']
        qty = math.floor(investmentAmount/price)
        daySIP = daySIP.append(
                    pd.Series(
                        [
                            date,
                            datetime.datetime.strptime(date,'%b %d, %Y').weekday(),
                            price,
                            qty,
                            qty*price
                        ],index=my_columns
                    ),ignore_index=True
                )
daySIP.to_csv('SIP Details/Day SIP.csv')